In [195]:
import gdxpds
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np

In [196]:
scenarios = list(['nodal', 'with_instrument', 'without_instrument']);
nodes = list(['north', 'south'])

In [197]:
scenarios = list(['nodal', 'with_instrument', 'without_instrument', 'agnostic_instrument']);

nodes = list(['north', 'south'])

In [198]:
def read_data(file, indicator):
    gams_dir='C:\Program Files\GAMS'
    df = gdxpds.to_dataframes(str(file) + '.gdx', gams_dir=gams_dir)
    df = df[indicator]
    df['model'] = file
    if 'Level' in df.columns:
        df['Value'] = df['Level']
    df['Value'] = df['Value'].round(2)
    return(df.set_index(['model']))

def read(indicator):
    if (len(scenarios) == 4):
        df = read_data(scenarios[0], indicator).append(read_data(scenarios[1], indicator)).append(read_data(scenarios[2], indicator)).append(read_data(scenarios[3], indicator))
    elif (len(scenarios) == 3):
        df = read_data(scenarios[0], indicator).append(read_data(scenarios[1], indicator)).append(read_data(scenarios[2], indicator))
    return(df)
    
def plot_distribution(df, axes, location):
    
    width = 0.8
    #baseload
    
    tmp = df.loc[df['tec'] == 'base'][location].fillna(0)
    tmp = tmp.reindex(index = scenarios)
    leg = axes.barh(y_pos, tmp, width, align='center', color = 'darkred')
    left = tmp.fillna(0)

    #peaker
    tmp = df.loc[df['tec'] == 'peak'][location].fillna(0)
    tmp = tmp.reindex(index = scenarios)
    axes.barh(y_pos, tmp, width, left=left, align='center', color = 'black')
    left = left + tmp.fillna(0)

    #wind
    tmp = df.loc[df['tec'] == 'wind'][location].fillna(0)
    tmp = tmp.reindex(index = scenarios)
    axes.barh(y_pos, tmp, width, left=left, align='center', color = 'lightblue')
    left = left + tmp.fillna(0)

    #solar
    tmp = df.loc[df['tec'] == 'solar'][location].fillna(0)
    tmp = tmp.reindex(index = scenarios)
    axes.barh(y_pos, tmp, width, left=left, align='center', color = 'gold')
    left = left + tmp.fillna(0)
    #leg.legend()
    return(axes)

    

In [199]:
df = read('o_gen')
df = df.reset_index().set_index(['model','t', 'n','tec'])
df = df.swaplevel().unstack()
df.columns = df.columns.droplevel(0)
df.columns.name = ''

df = df.groupby(['model','tec']).sum()
df = df.reset_index().set_index(['model'])
gen = df

In [200]:
gen

,tec,north,south
model,,,
agnostic_instrument,base,122.16,213.67
agnostic_instrument,peak,38.80,66.90
agnostic_instrument,solar,5.86,51.51
agnostic_instrument,wind,119.53,33.89
nodal,base,72.25,317.88
nodal,peak,69.73,19.27
nodal,solar,10.83,28.90
nodal,wind,94.33,35.97
with_instrument,base,98.27,277.99


In [201]:
gen['total'] = gen['north'] + gen['south']

In [202]:
gen['type'] = ['RE' if tec in ['wind', 'solar'] else 'non RE' for tec in gen['tec']]

In [203]:
re_shares = gen[['total', 'type']].groupby(['model', 'type']).sum()

In [204]:
re_shares = re_shares.unstack()

In [205]:
re_shares

total        
type                     RE  non RE
model                              
agnostic_instrument  210.79  441.53
nodal                170.03  479.13
with_instrument      174.89  477.59
without_instrument   257.51  395.24

In [206]:
re_shares['total', 'RE'] / (re_shares['total', 'RE'] + re_shares['total', 'non RE'])

model
agnostic_instrument    0.323139
nodal                  0.261923
with_instrument        0.268039
without_instrument     0.394500
dtype: float64

In [207]:
sums = gen.groupby(['model']).sum()
sums['total'] = sums[nodes[0]] + sums[nodes[1]]
sums = sums /48 * 8760
sums['share north'] = sums[nodes[0]] / sums['total']
sums
# in GWh

,north,south,total,share north
model,,,,
agnostic_instrument,52258.875,66789.525,119048.400,0.438972
nodal,45103.050,73368.650,118471.700,0.380707
with_instrument,45847.650,73229.950,119077.600,0.385023
without_instrument,66972.025,52154.850,119126.875,0.562191


# Installed capacities

In [208]:
df = read('o_cap')
df = df.reset_index().set_index(['model', 'n','tec'])
df = df.swaplevel().unstack()
df.columns = df.columns.droplevel(0)
df.columns.name = ''
df = df.reset_index().set_index(['model'])
capacities = df

In [209]:
capacities

,tec,north,south
model,,,
agnostic_instrument,base,10.18,18.71
agnostic_instrument,peak,7.21,15.74
agnostic_instrument,solar,4.25,34.34
agnostic_instrument,wind,48.59,16.44
nodal,base,6.17,27.47
nodal,peak,16.15,3.27
nodal,solar,7.85,19.26
nodal,wind,38.35,17.47
with_instrument,base,8.49,23.64


# Welfare comparison

In [210]:
welfare = pd.DataFrame(index = scenarios, columns = ['Network cost', 'Generation cost', 'Gross consumer surplus'])

In [211]:
for scenario in scenarios:
    welfare.loc[scenario, 'Network cost'] = read_data(scenario, 'network_cost').iloc[0,0]
    welfare.loc[scenario, 'Generation cost'] = read_data(scenario, 'generation_costs').iloc[0,0]
    welfare.loc[scenario, 'Gross consumer surplus'] = read_data(scenario, 'consumer_surplus').iloc[0,0]

In [212]:
welfare['Welfare'] = welfare['Gross consumer surplus'] - welfare['Network cost'] - welfare['Generation cost'];
#welfare['Welfare'] = welfare['Welfare']



In [213]:
welfare = welfare / 1000

In [214]:
welfare.style.format('{0:,.2f}')

,Network cost,Generation cost,Gross consumer surplus,Welfare
nodal,2.47,49.21,401.08,349.41
with_instrument,2.69,49.46,401.50,349.35
without_instrument,4.05,49.42,401.31,347.84
agnostic_instrument,3.33,49.29,401.34,348.72


In [215]:
print('welfare gains through instrument in %')
    
print(round((welfare['Welfare']['with_instrument'] - welfare['Welfare']['without_instrument']) / welfare['Welfare']['without_instrument'] * 100,2))

print('welfare gains through nodal pricing in %')
    
print(round((welfare['Welfare']['nodal'] - welfare['Welfare']['without_instrument']) / welfare['Welfare']['without_instrument'] * 100,2))

if len(scenarios) > 3:
    print('welfare gain through uniform signal in %')
    print(round((welfare['Welfare']['agnostic_instrument'] - welfare['Welfare']['without_instrument']) / welfare['Welfare']['with_instrument'] * 100,2))


welfare gains through instrument in %
0.43
welfare gains through nodal pricing in %
0.45
welfare gain through uniform signal in %
0.25


# Instrument level

In [216]:
fixed_costs = read_data('with_instrument', 'c_fix').reset_index(drop = True).set_index(['tec','n']).unstack()
fixed_costs.columns = fixed_costs.columns.droplevel(0)

In [217]:
fixed_costs

n,north,south
tec,,
base,411.0,411.0
peak,151.0,151.0
solar,82.0,82.0
wind,137.0,137.0


In [218]:
instr = read_data('with_instrument', 'o_instrument').reset_index(drop = True).set_index(['tec','n']).unstack()
instr.columns = instr.columns.droplevel(0)
instr = instr.round(1)

In [219]:
instr

n,north,south
tec,,
base,-3.2,-8.2
peak,-34.1,-32.2
solar,5.4,5.0
wind,4.1,-2.7


In [187]:
cap = read_data('with_instrument', 'o_cap').reset_index(drop = True).set_index(['tec','n']).unstack()
cap.columns = cap.columns.droplevel(0)
instr[cap.isna()] = np.nan

In [188]:
shares = round(100 *instr / fixed_costs,1)

In [189]:
shares = shares.rename(columns={'north':'north (share)', 'south': 'south (share)'})
#shares.columns = shares.columns.droplevel(0)

In [190]:
shares

n,north (share),south (share)
tec,,
base,-0.8,-2.0
peak,-22.6,-21.3
solar,6.6,6.1
wind,3.0,-2.0


In [191]:
instrument = pd.concat([instr, shares], axis=1)
instrument = instrument.sort_index(axis = 1)

In [192]:
#fixed_costs.columns = fixed_costs.columns.droplevel(0)
instrument['Fixed cost'] = fixed_costs['north']

In [193]:
format_dict = {'north':'{0:,.0f} €', 'north (share)': '{0:,.0f}%', 'south': '{0:,.0f} €', 'south (share)': '{0:,.0f}%', 'Fixed cost': '{0:,.0f} €'}
instrument.style.format(format_dict)

n,north,north (share),south,south (share),Fixed cost
tec,,,,,
base,-3 €,-1%,-8 €,-2%,411 €
peak,-34 €,-23%,-32 €,-21%,151 €
solar,5 €,7%,5 €,6%,82 €
wind,4 €,3%,-3 €,-2%,137 €


In [194]:
instr

n,north,south
tec,,
base,-3.2,-8.2
peak,-34.1,-32.2
solar,5.4,5.0
wind,4.1,-2.7
